In [1]:
import meshplot as mp
import numpy as np
import polyfempy as pf

In [2]:
def create_quad_mesh(n_pts):
    extend = np.linspace(0,1,n_pts)
    
    #x = extend n_pts times, y = each element from extend n_pts times 
    x, y = np.meshgrid(extend, extend, sparse=False, indexing='xy')
    
    #pts = every possible combination of x and y
    pts = np.column_stack((x.ravel(), y.ravel()))
    
    #i, j, k and l vals of the shape? 4 of them bc quad_mesh
    faces = np.ndarray([(n_pts-1)**2, 4],dtype=np.int32)

    index = 0
    for i in range(n_pts-1):
        for j in range(n_pts-1):
            faces[index, :] = np.array([
                j + i * n_pts,
                j+1 + i * n_pts,
                j+1 + (i+1) * n_pts,
                j + (i+1) * n_pts
            ])
            index = index + 1
            
    return pts, faces

In [3]:
mesh_path = "plane_hole.obj"

In [4]:
settings = pf.Settings(
    discr_order=1,
    pde=pf.PDEs.LinearElasticity
)

In [5]:
settings.set_material_params("E", 210000)
settings.set_material_params("nu", 0.3)

In [6]:
problem = pf.Problem()

In [7]:
problem.set_x_symmetric(1)
problem.set_y_symmetric(4)
problem.set_force(3, [100, 0])

In [8]:
settings.problem = problem

In [9]:
solver = pf.Solver()

solver.settings(settings)
solver.load_mesh_from_path(mesh_path, normalize_mesh=True)

[2023-06-02 17:58:24.037] [polyfem] [info] Loading mesh...
[2023-06-02 17:58:24.037] [geogram] [info] Loading file plane_hole.obj...
[2023-06-02 17:58:24.192] [geogram] [info] (FP64) nb_v:8549 nb_e:0 nb_f:16797 nb_b:299 tri:1 dim:3
[2023-06-02 17:58:24.192] [geogram] [info] Attributes on vertices: point[3]
[2023-06-02 17:58:24.245] [polyfem] [info] mesh bb min [0, 0], max [1, 0.500001]
[2023-06-02 17:58:24.246] [polyfem] [info]  took 0.209354s


In [10]:
solver.solve()

[2023-06-02 17:58:26.043] [polyfem] [info] simplex_count: 	16797
[2023-06-02 17:58:26.043] [polyfem] [info] regular_count: 	0
[2023-06-02 17:58:26.043] [polyfem] [info] regular_boundary_count: 	0
[2023-06-02 17:58:26.043] [polyfem] [info] simple_singular_count: 	0
[2023-06-02 17:58:26.043] [polyfem] [info] multi_singular_count: 	0
[2023-06-02 17:58:26.043] [polyfem] [info] boundary_count: 	0
[2023-06-02 17:58:26.043] [polyfem] [info] multi_singular_boundary_count: 	0
[2023-06-02 17:58:26.043] [polyfem] [info] non_regular_count: 	0
[2023-06-02 17:58:26.043] [polyfem] [info] non_regular_boundary_count: 	0
[2023-06-02 17:58:26.043] [polyfem] [info] undefined_count: 	0
[2023-06-02 17:58:26.043] [polyfem] [info] total count:	 16797
[2023-06-02 17:58:26.068] [polyfem] [info] Building isoparametric basis...
[2023-06-02 17:58:26.173] [polyfem] [info] Computing polygonal basis...
[2023-06-02 17:58:26.174] [polyfem] [info]  took 0.0011767s
[2023-06-02 17:58:26.179] [polyfem] [info] hmin: 0.00420

In [11]:
pts, tets, disp = solver.get_sampled_solution()

In [12]:
vertices = pts + disp

In [13]:
mises, _ = solver.get_sampled_mises_avg()

In [19]:
# mp.plot(vertices, tets, mises, return_plot=True)

mises

array([[132.50296492],
       [128.33963309],
       [124.17630126],
       ...,
       [146.39403661],
       [147.00204119],
       [145.97232228]])